In [2]:
import torch
import torchvision
import PIL
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from pathlib import Path
from PIL import Image

In [97]:
path = Path("E:\Projects\Ad_Classification")
train_dir = path / "train"
test_dir = path / "test"

train_dir, test_dir

(WindowsPath('E:/Projects/Ad_Classification/train'),
 WindowsPath('E:/Projects/Ad_Classification/test'))

In [98]:
train_transform = transforms.Compose(
    [transforms.Resize(size = (250, 250)),
    transforms.RandomRotation(degrees= 45),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor()])

test_transform = transforms.Compose(
    [transforms.Resize(size = (250, 250)),
    transforms.RandomRotation(degrees= 45),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor()])


In [99]:
train_data =  ImageFolder(root = train_dir, transform=train_transform, target_transform=None)
test_data =  ImageFolder(root = test_dir, transform=test_transform, target_transform=None)

In [100]:
train_data, test_data 

(Dataset ImageFolder
     Number of datapoints: 100
     Root location: E:\Projects\Ad_Classification\train
     StandardTransform
 Transform: Compose(
                Resize(size=(250, 250), interpolation=bilinear, max_size=None, antialias=warn)
                RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 24
     Root location: E:\Projects\Ad_Classification\test
     StandardTransform
 Transform: Compose(
                Resize(size=(250, 250), interpolation=bilinear, max_size=None, antialias=warn)
                RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                ToTensor()
            ))

In [101]:
train_data.classes, train_data.class_to_idx

(['Creative', 'Non_Creative'], {'Creative': 0, 'Non_Creative': 1})

In [55]:
data_loader = DataLoader(dataset=train_data, batch_size = 1, shuffle = True)


In [102]:
train_loader = DataLoader(dataset=train_data, batch_size = 1, shuffle = True)
test_loader = DataLoader(dataset=test_data, batch_size = 1, shuffle = False)

In [94]:
images, labels = next(iter(train_loader))

In [35]:
images[0], labels[0]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor(1))

In [62]:
images[0].shape

torch.Size([3, 250, 250])

In [36]:
import torch.nn as nn

In [103]:
class AdClassification(nn.Module):

    def __init__(self, input_shape : int, hidden_shape:int, output_shape : int) -> None:
        super(AdClassification, self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_shape*62*62, out_features=output_shape)
        )

    def forward(self, x:torch.Tensor):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.classifier(x2)
        return x3


In [86]:
model = AdClassification(3, 10, len(train_data.classes))

model.state_dict()

OrderedDict([('block1.0.weight',
              tensor([[[[-0.0512, -0.0246,  0.0658],
                        [-0.0304,  0.0175,  0.1164],
                        [-0.1135,  0.0833, -0.1072]],
              
                       [[ 0.0795, -0.1840, -0.0950],
                        [ 0.1157, -0.1465,  0.1121],
                        [-0.0176, -0.0995,  0.0020]],
              
                       [[ 0.0775, -0.1762, -0.1618],
                        [-0.0034,  0.1488, -0.0866],
                        [-0.1846,  0.0924,  0.0110]]],
              
              
                      [[[ 0.1521,  0.1677, -0.0235],
                        [ 0.1705, -0.1769,  0.0194],
                        [ 0.1428,  0.0513,  0.0972]],
              
                       [[-0.0814,  0.1008, -0.0197],
                        [-0.0699,  0.1398, -0.0385],
                        [ 0.1810, -0.0935,  0.0920]],
              
                       [[-0.0891, -0.0426,  0.0287],
                       

In [60]:
model.parameters()

<generator object Module.parameters at 0x00000262CAB60AC0>

In [87]:
optimizer  = torch.optim.SGD(params = model.parameters(), lr = 0.01)
loss_fn = nn.CrossEntropyLoss()

In [109]:
def train_step(model: nn.Module, 
               train_loader : torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(train_loader):

        pred_y = model(X)
        loss = loss_fn(pred_y, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_class = torch.argmax(torch.softmax(pred_y, dim = 1), dim = 1)
        train_acc += (pred_class == y).sum().item()/len(pred_y)

    train_loss = train_loss/len(train_loader)
    train_acc = train_acc/len(train_loader)

    return train_loss, train_acc 

In [47]:
def test_step(
        model: nn.Module,
        dataloader : torch.utils.data.DataLoader,
        loss_fn : torch.nn.Module):
    
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():

        for batch, (X,y) in enumerate(dataloader):

            pred_y = model(X)
            loss = loss_fn(pred_y, y)
            test_loss += loss.item()

            pred_class = torch.argmax(torch.softmax(pred_y, dim =1), dim =1)
            test_acc += (pred_class == y).sum().item()/len(pred_y)

        test_loss = test_loss/len(dataloader)
        test_acc = test_acc/len(dataloader)

        return test_loss, test_acc

In [105]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_loader: torch.utils.data.DataLoader, 
          test_loader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_loader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        
        test_loss, test_acc = test_step(model=model,
            dataloader=test_loader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
train(model, train_loader, test_loader, optimizer, loss_fn)

In [ ]:
Model_path = Path("AdClassificationModel")
Model_path.mkdir(parents = True, exist_ok = True)

Model_Name  = "AdDetection.pt"

SaveModelPath = Model_path / Model_Name

SaveModelPath

In [ ]:
torch.save(model.state_dict(), SaveModelPath)

In [130]:
from PIL import Image
pic = Image.open(r"E:\Projects\Ad_Classification\test\Creative\15 Extremely Creative Durex Condom Ads.jpeg")
test_image = train_transform(pic)

test_image.shape

torch.Size([3, 250, 250])

In [ ]:
imported = AdClassification()

imported.load_state_dict(torch.load(SaveModelPath))
imported.eval()

with torch.inference_mode():
    loaded_pred = imported(test_image)
    pred_class = torch.argmax(torch.softmax(loaded_pred, dim =1), dim =1)
pred_class
